In [48]:
import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"
import plotly.express as px
import plotly.graph_objects as go

dataset_link_dict = {'Mass Flux': "https://raw.githubusercontent.com/jtarriela/FDA_Blood_Pump/mesh_convergence_study/Core/mesh_convergence_study/Convergence%20Data/Residual%20Data/Coarse/mass-flux.out",
                     'Max Velocity': "https://raw.githubusercontent.com/jtarriela/FDA_Blood_Pump/mesh_convergence_study/Core/mesh_convergence_study/Convergence%20Data/Residual%20Data/Coarse/max-vel-rotor.out",
                     'Total Pressure In': "https://raw.githubusercontent.com/jtarriela/FDA_Blood_Pump/mesh_convergence_study/Core/mesh_convergence_study/Convergence%20Data/Residual%20Data/Coarse/pressure-in.out",
                     'Total Pressure Out': "https://raw.githubusercontent.com/jtarriela/FDA_Blood_Pump/mesh_convergence_study/Core/mesh_convergence_study/Convergence%20Data/Residual%20Data/Coarse/pressure-out.out",
                     'Q1 Velocity': "https://raw.githubusercontent.com/jtarriela/FDA_Blood_Pump/mesh_convergence_study/Core/mesh_convergence_study/Convergence%20Data/Quantity%20Data/Coarse/vel-q1",
                     'Q2 Velocity': "https://raw.githubusercontent.com/jtarriela/FDA_Blood_Pump/mesh_convergence_study/Core/mesh_convergence_study/Convergence%20Data/Quantity%20Data/Coarse/vel-q2"}

mass_flux_df = pd.read_csv(dataset_link_dict['Mass Flux'], header = 1 , skiprows =[2], delim_whitespace = True)
max_vel_rotor_df = pd.read_csv(dataset_link_dict['Max Velocity'], header = 1 , skiprows =[2], delim_whitespace = True)
total_p_in = pd.read_csv(dataset_link_dict['Total Pressure In'], header = 1 , skiprows =[2], delim_whitespace = True)
total_p_out = pd.read_csv(dataset_link_dict['Total Pressure Out'], header = 1 , skiprows =[2], delim_whitespace = True)

Q1_velocity_df = pd.read_csv(dataset_link_dict['Q1 Velocity'], header = 1 , skiprows =[1], delim_whitespace = True)
Q2_velocity_df = pd.read_csv(dataset_link_dict['Q2 Velocity'], header = 1 , skiprows =[1], delim_whitespace = True)

Q1_velocity_df = Q1_velocity_df.rename(columns ={'((xy/key/label':'Position (m)','quadrant-1)':'Velocity Magnitude (m/s)'})
Q2_velocity_df = Q2_velocity_df.rename(columns ={'((xy/key/label':'Position (m)','quadrant-2)':'Velocity Magnitude (m/s)'})
Q1_velocity_df.dropna(subset = ['Velocity Magnitude (m/s)'], inplace=True)
Q2_velocity_df.dropna(subset = ['Velocity Magnitude (m/s)'], inplace=True)

In [49]:
with pd.option_context('display.precision', 10):
    print(total_p_out)

      Iteration          p-out
0             1  70.3058692241
1             2  70.2119790574
2             3  70.2893933252
3             4  70.4278261753
4             5  70.6186478866
...         ...            ...
5195       5196  97.7906314491
5196       5197  97.7906314491
5197       5198  97.7906314491
5198       5199  97.7906314491
5199       5200  97.7906314491

[5200 rows x 2 columns]


# Mesh Convergence: Coarse Simulation <br>
See Readme for simulation conditions.

### Mass Flow Residuals

In [50]:
fig = px.line(mass_flux_df,x ='Iteration',y='mass-flux')
fig.show()

Steady state simulations appear to oscillate with bounds around |5E-3|.<br>
Inherent instability visualized due to the transient nature of the problem.

### Pressure Residuals

In [51]:
density = 1056 #kg/m3
gravity = 9.81 #m/s2
total_pressure_df = pd.DataFrame()
total_pressure_df['Iteration'] = total_p_in["Iteration"]
total_pressure_df['P_in (Pa)'] = total_p_in["p-in"]
total_pressure_df['P_out (Pa)'] = total_p_out["p-out"]
total_pressure_df['P_head (m)'] = (total_pressure_df['P_out (Pa)'] - total_pressure_df['P_in (Pa)'])/(density*gravity)
total_pressure_df['P_head (mmHg)'] = total_pressure_df['P_head (m)'] * 73.53708129594501
# fig = px.line(total_pressure_df,x =['Iteration','Iteration'],y=['P_in (Pa)]','P_out (Pa)'])
# fig.show()
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=total_pressure_df['Iteration'],
                          y=total_pressure_df['P_in (Pa)'],
                          mode='lines+markers',
                          name='Inlet Pressure (Pa)'))
fig1.update_layout(
    title="Total Inlet Pressure Monitor Values",
    xaxis_title="Iteration",
    yaxis_title="Pressure (Pa)",
    legend_title=None,
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="Black"))
fig1.show()

fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=total_pressure_df['Iteration'],
                          y=total_pressure_df['P_out (Pa)'],
                          mode='lines+markers',
                          name='Outlet Pressure (Pa)'))
fig2.update_layout(
    title="Total Outlet Pressure Monitor Values",
    xaxis_title="Iteration",
    yaxis_title="Pressure (Pa)",
    legend_title=None,
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="Black"))
fig2.show()

fig3 = go.Figure()
fig3.add_trace(go.Scatter(x=total_pressure_df['Iteration'],
                          y=total_pressure_df['P_head (mmHg)'],
                          mode='lines+markers',
                          name='Pressure Head (mmHg)'))
fig3.update_layout(
    title="Computed Pressure Head per Iteration",
    xaxis_title="Iteration",
    yaxis_title="Pressure Head (mmHg)",
    legend_title=None,
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="Black"))
fig3.show()

# fig1 = go.Figure()
# fig1.add_trace(go.Scatter(x=total_pressure_df['Iteration'],
#                           y=total_pressure_df['P_in (Pa)'],
#                           mode='markers',
#                           name='Inlet Pressure (Pa)'))
# fig1.add_trace(go.Scatter(x=total_pressure_df['Iteration'],
#                           y=total_pressure_df['P_out (Pa)'],
#                           mode='lines+markers',
#                           name='Outlet Pressure (Pa)'))
# fig1.add_trace(go.Scatter(x=total_pressure_df['Iteration'],
#                           y=total_pressure_df['P_head (m)'],
#                           mode='lines',
#                           name='Outlet Pressure (Pa)'))
#
# fig1.update_layout(
#     title="Plot Title",
#     xaxis_title="X Axis Title",
#     yaxis_title="Y Axis Title",
#     legend_title="Legend Title",
#     font=dict(
#         family="Courier New, monospace",
#         size=18,
#         color="RebeccaPurple"))
# fig1.show()


### Rotor Domain Max Velocity Monitor

In [52]:
fig4 = go.Figure()
fig4.add_trace(go.Scatter(x=max_vel_rotor_df['Iteration'],
                          y=max_vel_rotor_df['max-vel-rotor'],
                          mode='lines+markers',
                          name='Velocity Magnitude (m/s)'))
fig4.update_layout(
    title="Max Velocity Magnitude in Rotor Domain Monitor",
    xaxis_title="Iteration",
    yaxis_title="Velocity Magnitude (m/s)",
    legend_title=None,
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="Black"))
fig4.show()

### Velocity Line Surface Data

In [54]:
fig5 = go.Figure()
fig5.add_trace(go.Scatter(x=Q1_velocity_df['Position (m)'],
                          y=Q1_velocity_df['Velocity Magnitude (m/s)'],
                          mode='lines+markers',
                          name='Velocity Magnitude (m/s)'))
fig5.update_layout(
    title="Coarse Quadrant 1 Velocity Profile",
    xaxis_title='Position (m)',
    yaxis_title="Velocity Magnitude (m/s)",
    legend_title=None,
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="Black"))
fig5.show()

fig6 = go.Figure()
fig6.add_trace(go.Scatter(x=Q2_velocity_df['Position (m)'],
                          y=Q2_velocity_df['Velocity Magnitude (m/s)'],
                          mode='lines+markers',
                          name='Velocity Magnitude (m/s)'))
fig6.update_layout(
    title="Coarse Quadrant 2 Velocity Profile",
    xaxis_title='Position (m)',
    yaxis_title="Velocity Magnitude (m/s)",
    legend_title=None,
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="Black"))
fig6.show()